В стандартной установке может отсутствовать часть модулей, необходимых для работы, их нужно установить  
Команда установки "pip install имя_модуля" вводится в этом же окне.  
После установки необходимо перезапустить ядро выбрав в меню сверху Kernel-Restart. 

In [ ]:
import pandas as pd #Для работы с таблицами данных (дата фреймы)
import requests #Для запросов к серверу
import json #Для обработки ответов сервера

# Выгрузка данных по акции Сбербанка

Сперва зададим изначально необходимые значения  
Сформируем базовый заспрос и пустой дата фрейм с именами колонок

In [ ]:
# В базовом запросе мы укажем:
# engines/stok (фондовый рынок), markets/shares - рынок акций, boards/TQBR - режим торгов для акции сбербанка TQBR
# Базовый запрос поместим в переменную base_url
base_url = "http://iss.moex.com/iss/history/engines/stock/markets/shares/boards/TQBR/securities.json"
response = requests.get(base_url) # Получим ответ от сервера 
result = json.loads(response.text) # Расшифровываем ответ от сервера
col_name = result['history']['columns'] # Задаем имена колонок извлекая данные из ответа сервера
data_shares = pd.DataFrame(columns = col_name) #

In [ ]:
data_shares  # Можно посмотреть что получилось

In [ ]:
url_share = 'http://iss.moex.com/iss/history/engines/stock/markets/shares/boards/TQBR/securities/SBER.json' #Первая страница данных
# Получаем данные
response = requests.get(url_share)
result = json.loads(response.text) 
resp_date = result['history']['data'] 
#Извлеченные данные добавляем в начало дата фрейма
data_shares  = pd.DataFrame(resp_date, columns = col_name)
# Узнаем количество полученных строк, если их меньше 100, то мы получили все данные
a = len(resp_date)

Если же значение а равно 100, то получены не все данные по выбранной бумаге.  
В этом случае запускаем в цикле переборку остальных страниц.  
Цикл будет работать до тех пор, пока мы не получим количество строк меньше, чем 100.  
Для этого каждый раз на предпоследнем шаге цикла переопределяем переменную.

In [ ]:
b = 100
while a == 100:
    url_opt = '?start=' + str(b)
    url_next_page  = url_share + url_opt
    response = requests.get(url_next_page)
    result = json.loads(response.text)
    resp_date = result['history']['data']
    data_next_page = pd.DataFrame(resp_date, columns = col_name)
    data_shares = pd.concat([data_shares, data_next_page], ignore_index=True) 
    a = len(resp_date)
    b = b + 100

In [ ]:
b

Судя по переменной b, у нас должно быть не менее 2400 строк, проверим

In [ ]:
data_shares

Мы получили 2405 строк, с датами с 25 марта 2013 г. по предыдущую торговую дату

# Облигации

Данные по облигациям собираются аналогичным образом, изменяется лишь базовый запрос.  
https://iss.moex.com/iss/history/engines/stock/markets/bonds/boards/TQCB/securities  
Как видно из базового запроса, у нас изменился параметр после markets - shares поменяли на bonds и значение режима идентификатора торгов с TQBR на TQCB
Найти название нужной облигаии можно на сайте московской биржи в разделе "Биржевая информация". Выбрав нужный раздел и бумагу, мы увидим необходимую информацию

Возьмем облигацию РЖД-32 обл. Ее код RU000A0JSGV0, идентификатор режима торгов TQCB  
Кроме этого, можно получить информацию об инструменте и через API, указав в запросе код бумаги
https://iss.moex.com/iss/securities/RU000A0JSGV0

In [ ]:
base_url = "http://iss.moex.com/iss/history/engines/stock/markets/bonds/boards/TQCB/securities.json" #Задаем базовый адрес запроса для облигаций
response = requests.get(base_url) #Получим ответ от сервера 
result = json.loads(response.text)
col_name = result['history']['columns'] #Задаем имена колонок извлекая данные из ответа сервера
data_bonds = pd.DataFrame(columns = col_name)

Проверим полученые колонки

In [ ]:
data_bonds

In [ ]:
url_bonds = 'http://iss.moex.com/iss/history/engines/stock/markets/bonds/boards/TQCB/securities/RU000A0JSGV0.json' #Первая страница данных
# Получаем данные
response = requests.get(url_bonds)
result = json.loads(response.text)
resp_date = result['history']['data'] 
#Извлеченные данные добавляем в начальный дата фрейм
data_bonds  = pd.DataFrame(resp_date, columns = col_name)
# Узнаем количество полученных строк, если их меньше 100, то мы получили все данные
a = len(resp_date)

Проверим первые полученные данные

In [ ]:
data_bonds

На сайте мосбиржи для этой бумаги указана дата начала торгов 20.07.2012, однако мы получили данные только с 25 мая 2020 года. 
Открыв на сайте мос биржи на странице этой бумаги вкладку "Режимы торгов", можно увидеть, что в указанном режиме торгов бумага торгуется как раз с 25.05.2020, однако никакой информации за более ранние даты даже по другим режимам торгов информация через API и на сайте биржи нет

Как и в примере с акцией Сбера, собираем в цикле данные по облигации РЖД

In [ ]:
b = 100
while a == 100:
    url_opt = '?start=' + str(b)
    url_next_page  = url_bonds + url_opt
    response = requests.get(url_next_page)
    result = json.loads(response.text)
    resp_date = result['history']['data']
    data_next_page = pd.DataFrame(resp_date, columns = col_name)
    data_bonds  = pd.concat([data_bonds, data_next_page], ignore_index=True) 
    a = len(resp_date)
    b = b + 100

Проверяем, что получилось

In [ ]:
data_bonds

Как мы видим, были выгружены данные с 25 мая 2020 г. по предыдущую торговую дату

# Другие инструменты

Попробуем выгрузить аналогичную информацию по другим инструментам мосбиржи  
Например индекс мосбиржи IMOEX  
Сперва получим информацию о бумаге: http://iss.moex.com/iss/securities/IMOEX  
Получим следующую базовую информацию:  
engine - stock  
market  - index  
boardid - SNDX  
Составляем базовый запрос:
http://iss.moex.com/iss/history/engines/stock/markets/bonds/boards/SNDX/securities.json  
Запрос для выбранной бумаги: 
http://iss.moex.com/iss/history/engines/stock/markets/index/boards/SNDX/securities/imoex.json  
Как видно из ответа, по индексу IMOEX доступны данные начиная с 22 сентября 1997 года  
Скачаем данные аналогично другим инструментам

In [ ]:
base_url = "http://iss.moex.com/iss/history/engines/stock/markets/index/boards/SNDX/securities.json" #Задаем базовый адрес запроса для облигаций
response = requests.get(base_url) #Получим ответ от сервера 
result = json.loads(response.text)
col_name = result['history']['columns'] #Задаем имена колонок извлекая данные из ответа сервера
data_index = pd.DataFrame(columns = col_name)

In [ ]:
data_index

In [ ]:
url_index = 'http://iss.moex.com/iss/history/engines/stock/markets/index/boards/SNDX/securities/imoex.json' #Первая страница данных
# Получаем данные
response = requests.get(url_index)
result = json.loads(response.text)
resp_date = result['history']['data'] 
#Извлеченные данные добавляем в начальный дата фрейм
data_index  = pd.DataFrame(resp_date, columns = col_name)
# Узнаем количество полученных строк, если их меньше 100, то мы получили все данные
a = len(resp_date)

In [ ]:
data_index

In [ ]:
b = 100
while a == 100:
    url_opt = '?start=' + str(b)
    url_next_page  = (url_index + url_opt)
    response = requests.get(url_next_page)
    result = json.loads(response.text)
    resp_date = result['history']['data']
    data_next_page = pd.DataFrame(resp_date, columns = col_name)
    data_index  = pd.concat([data_index, data_next_page], ignore_index=True) 
    a = len(resp_date)
    b = b + 100

In [ ]:
data_index

В итоге мы за несколько секунд получили 6254 строки или данные по инструменту с 22 сентября 1997 г. по последнюю торговую дату

Все полученные данные можно сохранить в файл формата csv для последующей обработки удобным методом

In [ ]:
data_shares.to_csv('SBER_22.csv', encoding='cp1251')
data_bonds.to_csv('RZD_22.csv', encoding='cp1251')
data_index.to_csv('IMOEX_22.csv', encoding='cp1251', sep=';')

Синтаксис сохранения в файл простой  
имя_набора данных.to_csv('имя файла с расширением', encoding='кодировка_файла', sep='разделитель')  
Параметр кодировки необязательный, но при работе с кирилицей лучше использовать его  
Параметр разделителя - по умолчанию это ','. Если использовать sep=';', то данные при открытии csv файла Excel'ем будут уже разбиты на столбы, иначе вы получите данные одной строки в одном столбце, значения в котором будут разделены запятой  
Если необходимо сохранить файл в определенное место на компьютере, а не в рабочей папке проект, то имя файла указывается в следующем формате: data.to_csv(r'D:\имя_папки\имя_файла.csv', encoding='cp1251')'
